In [1]:
!pip install gensim==4.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
%%time

# Importing necessary libraries

import gensim.downloader
from gensim.models import Word2Vec
from gensim.parsing.preprocessing import preprocess_string
import json
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from wordcloud import WordCloud, ImageColorGenerator

/usr/local/lib/python3.9/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


CPU times: user 960 ms, sys: 228 ms, total: 1.19 s
Wall time: 1.08 s


In [3]:
%%time


'''
Reading from input file and doing json normalizing upon the train data to achieve the required 
dataframe format
'''
def convert_from_json_to_dataframe(file_path, record_path=['data', 'paragraphs', 'qas', 'answers']):
    file = json.loads(open(file_path).read())
    js = pd.json_normalize(file, record_path)
    m = pd.json_normalize(file, record_path[:-1])
    r = pd.json_normalize(file, record_path[:-2])
    index = np.repeat(r['context'].values, r.qas.str.len())
    m['context'] = index
    data = m[['id', 'question', 'context', 'answers']].set_index('id').reset_index()
    data['context_id'] = data['context'].factorize()[0]
    return data
    

train_file_path = '/content/train-v1.1.json'
train_data = convert_from_json_to_dataframe(train_file_path)
train_data

CPU times: user 4.26 s, sys: 172 ms, total: 4.43 s
Wall time: 4.52 s


,id,question,context,answers,context_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.","[{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}]",0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.","[{'answer_start': 188, 'text': 'a copper statue of Christ'}]",0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame is beside to which structure?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.","[{'answer_start': 279, 'text': 'the Main Building'}]",0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.","[{'answer_start': 381, 'text': 'a Marian place of prayer and reflection'}]",0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre Dame?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, Franc

In [4]:
%%time

# Exploring the train dataset a bit more!

print ('Number of unique contexts in the train dataset :', len(train_data['context_id'].unique()))

Number of unique contexts in the train dataset : 18891
CPU times: user 1.56 ms, sys: 957 µs, total: 2.52 ms
Wall time: 2.7 ms


In [5]:
%%time

# Getting unique documents from the train dataset

train_unique_documents = train_data[['context', 'context_id']].drop_duplicates().reset_index(drop=True)
train_unique_documents

CPU times: user 137 ms, sys: 2.93 ms, total: 140 ms
Wall time: 147 ms


,context,context_id
0,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",0
1,"As at most other universities, Notre Dame's students run a number of news media outlets. The nine student-run outlets include three newspapers, both a radio and television station, and several magazines and journals. Begun as a one-page journal in September 1876, the Scholastic magazine is issued twice monthly and claims to be the oldest continuous collegiate publication in the United States. The other magazine, The Juggler, is released twice a year and focuses on student literature and artwork. The Dome yearbook is published annually. The newspapers have varying publication interests, with The Observer published daily and mainly reporting university and other news, and staffed by students from both Notre Dame and Saint Mary's College. Unlike Scholastic and The Dome, The Observer is an independent publication and does not have a faculty advisor or any editorial oversight from the University. In 1987, when some students believed that The Observer began to show a conservative bias, a liberal newspaper, Common Sense was published. Likewise, in 2003, when other students believed that the paper showed a liberal bias, the conservative paper Irish Rover went into production. Neither paper is published as often as The Observer; however, all three are distributed to all students. Finally, in Spring 2008 an undergraduate journal for political science research, Beyond Politics, made its debut.",1
2,"The university is the major seat of the Congregation of Holy Cross (albeit not its official headquarters, which are in Rome). Its main seminary, Moreau Seminary, is located on the campus across St. Joseph lake from the Main Building. Old College, the oldest building on campus and located near the shore of St. Mary lake, houses undergraduate seminarians. Retired priests and brothers reside in Fatima House (a former retreat center), Holy Cross House, as well as Columba Hall near the Grotto. The university through the Moreau Seminary has ties to theologian Frederick Buechner. While not Catholic, Buechner has praised writers from Notre Dame and Moreau Seminary created a Buechner Prize for Preaching.",2
3,"The College of Engineering was established in 1920, however, early courses in civil and mechanical engineering were a part of the College of Science since the 1870s. Today the college, housed in the Fitzpatrick, Cushing, and Stinson-Remick Halls of Engineering, includes five departments of study – aerospace and mechanical engineering, chemical and biomolecular engineering, civil engineering and geological sciences, computer science and engineering, and electrical engineering – with eight B.S. degrees offered. Additionally, the college offers five-year dual degree programs with the Colleges of Arts and Letters and of Business awarding additional B.A. and Master of Business Administration (MBA) degrees, respectively.",3
4,"All of Notre Dame's undergraduate students are a part of one of the five undergraduate colleges at the school or are in the First Year of Studies program. The First Year of Studies program was established in 1962 to guide incoming freshmen in their first year at the school before they have declared a major. Each student is given an academic advisor from the program who helps them to choose classes that give them exp

In [6]:
%%time

# Creating a corpus of tokens

train_corpus = train_unique_documents['context'].tolist()
train_corpus = [preprocess_string(t) for t in train_corpus]

CPU times: user 16.6 s, sys: 97.8 ms, total: 16.7 s
Wall time: 20.4 s


In [7]:
%%time

# Defining the Word2Vec model

vectorizer = Word2Vec(sentences=train_corpus, vector_size=5000, window=5, min_count=1, workers=4, epochs=2).wv

CPU times: user 1min 5s, sys: 3.3 s, total: 1min 8s
Wall time: 40.7 s


In [8]:
%%time

# Getting the 5 most similar words to a sample word

vectorizer.most_similar('tourist', topn=5)

CPU times: user 458 ms, sys: 241 ms, total: 699 ms
Wall time: 608 ms


[('larg', 0.9999658465385437),
 ('near', 0.9999656081199646),
 ('compani', 0.9999654293060303),
 ('includ', 0.999965250492096),
 ('main', 0.9999650716781616)]

In [9]:
%%time

# Text Transformation 
def text_transformation(vectorizer, text, verbose=True):
    tokens = preprocess_string(text)
    words = [vectorizer[w] for w in tokens if w in vectorizer]
    vector = [w for w in tokens if w in vectorizer]
    if verbose:
        print ('Text :', text)
        print ('Vector :', vector)
    elif len(words):
        return np.mean(words, axis=0)
    else:
        return np.zeros((5000), dtype=np.float32)
    

question = 'What are the tourist spots in Spain?'    
text_transformation(vectorizer, question, True)

Text : What are the tourist spots in Spain?
Vector : ['tourist', 'spot', 'spain']
CPU times: user 1.7 ms, sys: 7.01 ms, total: 8.71 ms
Wall time: 7.39 ms


In [10]:
%%time

# Training the model

retriever_configs = {'n_neighbors' : 3, 'metric' : 'cosine'}
retriever = NearestNeighbors(**retriever_configs)
X_train = train_unique_documents['context'].apply(lambda x : text_transformation(vectorizer, x, False)).tolist()
retriever.fit(X_train, train_unique_documents['context_id'])

CPU times: user 24.7 s, sys: 364 ms, total: 25.1 s
Wall time: 25.7 s


NearestNeighbors(metric='cosine', n_neighbors=3)

In [11]:
%%time

# Evaluating the above model

X_train = train_data['question'].apply(lambda x : text_transformation(vectorizer, x, False)).tolist()

# predict one document for each question
y_test = train_data['context_id']
y_pred = retriever.kneighbors(X_train, return_distance=False)

CPU times: user 8min 4s, sys: 16.4 s, total: 8min 21s
Wall time: 5min 6s


In [12]:
%%time

y_pred

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


array([[ 8472, 13118, 17853],
       [11436, 12520, 15605],
       [14064,  2092, 16403],
       ...,
       [ 3820,  5641,  2731],
       [ 7695,  2277, 14333],
       [10282,  4353,  8174]])

In [13]:
%%time

# Testing the model upon the training set
def compute_accuracy(y_test, y_pred):
    num_correct, num_total = 0, len(y_test)
    for i in range(num_total):
        y_pred_list = y_pred[i]
        if y_test[i] in y_pred_list:
            num_correct += 1
    return num_correct / num_total


acc = compute_accuracy(y_test, y_pred)
print('Accuracy:', f'{acc:.4f}')

Accuracy: 0.0390
CPU times: user 681 ms, sys: 36 µs, total: 681 ms
Wall time: 683 ms
